## 训练深度神经网络

### 梯度消失与梯度爆炸问题

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.models.Sequential([
    layers.Flatten(input_shape=[28, 28]),
    layers.BatchNormalization(),
    layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dense(10, activation='softmax'),
])

### 重用预训练层

In [4]:
import numpy as np

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [5]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))
model_A.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(lr=1e-3), metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=20, validation_data=(X_valid_A, y_valid_A))
model_A.save("my_model_A.h5")

/usr/local/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
Epoch 1/20
1375/1375 [==============================] - 7s 4ms/step - loss: 0.5472 - accuracy: 0.8264 - val_loss: 0.3763 - val_accuracy: 0.8717
Epoch 2/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3507 - accuracy: 0.8790 - val_loss: 0.3277 - val_accuracy: 0.8884
Epoch 3/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.3162 - accuracy: 0.8901 - val_loss: 0.3062 - val_accuracy: 0.8954
Epoch 4/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2970 - accuracy: 0.8972 - val_loss: 0.2904 - val_accuracy: 0.9041
Epoch 5/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2840 - accuracy: 0.9024 - val_loss: 0.2811 - val_accuracy: 0.9041
Epoch 6/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2746 - accura

In [6]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))
model_B.compile(loss="binary_crossentropy", optimizer=keras.optimizers.SGD(lr=1e-3), metrics=["accuracy"])
history = model_B.fit(X_train_B, y_train_B, epochs=20, validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 2s 55ms/step - loss: 0.8249 - accuracy: 0.3850 - val_loss: 0.6746 - val_accuracy: 0.6065
Epoch 2/20
7/7 [==============================] - 0s 21ms/step - loss: 0.6148 - accuracy: 0.6950 - val_loss: 0.5343 - val_accuracy: 0.7546
Epoch 3/20
7/7 [==============================] - 0s 26ms/step - loss: 0.4845 - accuracy: 0.8200 - val_loss: 0.4402 - val_accuracy: 0.8854
Epoch 4/20
7/7 [==============================] - 0s 20ms/step - loss: 0.3999 - accuracy: 0.9200 - val_loss: 0.3735 - val_accuracy: 0.9391
Epoch 5/20
7/7 [==============================] - 0s 19ms/step - loss: 0.3382 - accuracy: 0.9450 - val_loss: 0.3233 - val_accuracy: 0.9675
Epoch 6/20
7/7 [==============================] - 0s 50ms/step - loss: 0.2926 - accuracy: 0.9600 - val_loss: 0.2863 - val_accuracy: 0.9746
Epoch 7/20
7/7 [==============================] - 0s 18ms/step - loss: 0.2574 - accuracy: 0.9750 - val_loss: 0.2559 - val_accuracy: 0.9777
Epoch 8/20
7/7 [===========

In [7]:
model_A = keras.models.load_model('my_model_A.h5')
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation='sigmoid'))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy", optimizer='sgd', metrics=['accuracy'])

In [10]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True
optimizer = keras.optimizers.SGD(learning_rate=1e-4)
model_B_on_A.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))
print(model_B_on_A.evaluate(X_test_B, y_test_B))

Epoch 1/4
7/7 [==============================] - 0s 23ms/step - loss: 0.0890 - accuracy: 1.0000 - val_loss: 0.1028 - val_accuracy: 0.9939
Epoch 2/4
7/7 [==============================] - 0s 30ms/step - loss: 0.0882 - accuracy: 1.0000 - val_loss: 0.1020 - val_accuracy: 0.9939
Epoch 3/4
7/7 [==============================] - 0s 29ms/step - loss: 0.0874 - accuracy: 1.0000 - val_loss: 0.1011 - val_accuracy: 0.9939
Epoch 4/4
7/7 [==============================] - 0s 19ms/step - loss: 0.0866 - accuracy: 1.0000 - val_loss: 0.1003 - val_accuracy: 0.9939
Epoch 1/16
7/7 [==============================] - 1s 44ms/step - loss: 0.0858 - accuracy: 1.0000 - val_loss: 0.0995 - val_accuracy: 0.9939
Epoch 2/16
7/7 [==============================] - 0s 18ms/step - loss: 0.0850 - accuracy: 1.0000 - val_loss: 0.0987 - val_accuracy: 0.9939
Epoch 3/16
7/7 [==============================] - 0s 19ms/step - loss: 0.0843 - accuracy: 1.0000 - val_loss: 0.0980 - val_accuracy: 0.9939
Epoch 4/16
7/7 [===============

### 更快的优化器

### 通过正则化避免过拟合

In [11]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense, activation='elu', kernel_initializer='he_normal', kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation='softmax', kernel_initializer='glorot_uniform'),
])

In [13]:
 model = keras.models.Sequential([
     layers.Flatten(input_shape=[28, 28]),
     layers.Dropout(rate=0.2),
     layers.Dense(300, activation='relu', kernel_initializer='he_normal'),
     layers.Dropout(rate=0.2),
     layers.Dense(100, activation='relu', kernel_initializer='he_normal'),
     layers.Dropout(rate=0.2),
     layers.Dense(10, activation='softmax'),
 ])

In [ ]:
class MCDropout(layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)